## Load packages and data

In [18]:
# Loading the packages to be used
from __future__ import print_function  # Python 2 and 3
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [23]:
#pd.set_option("display.max_rows", 400)
pd.set_option("display.max_columns", 400)

In [24]:
# Import data; for linear regression, either import encoded datasets or dummify and then oneHotEncode in python
houses_train = pd.read_csv('/Users/kathrynbryant/Dropbox/NYCDSA/blackFlamingo/Data/encoded_houses_train.csv')
houses_test = pd.read_csv('/Users/kathrynbryant/Dropbox/NYCDSA/blackFlamingo/Data/encoded_houses_test.csv')
encoded_private_train = pd.read_csv('/Users/kathrynbryant/Dropbox/NYCDSA/blackFlamingo/Data/encoded_private_train.csv')
encoded_private_test = pd.read_csv('/Users/kathrynbryant/Dropbox/NYCDSA/blackFlamingo/Data/encoded_private_test.csv')

## Define predictor and response objects

In [25]:
# Define data frame of predictors and BoxCox response variable
X_private_train = encoded_private_train.drop('SalePrice', axis = 1)
#yT_private_train, private_lambda = stats.boxcox(encoded_private_train[['SalePrice']]) # T indicates transformation; ndarray 
yT_private_train = stats.boxcox(encoded_private_train['SalePrice'], lmbda = 0.22)
# Note: The above lambda was found in R; to let Python find it, use the commented-out line above

X_private_test = encoded_private_test.drop('SalePrice', axis = 1)

X_houses_train = houses_train.drop('SalePrice', axis = 1)
yT_houses_train = stats.boxcox(houses_train['SalePrice'], lmbda = 0.22) # T indicates transformation; ndarray output

In [26]:
X_private_train.head()

,MSSubClass_lev_x.120,MSSubClass_lev_x.160,MSSubClass_lev_x.190,MSSubClass_lev_x.30,MSSubClass_lev_x.50,MSSubClass_lev_x.60,MSSubClass_lev_x.90,MSSubClass_catP,MSSubClass_catN,MSSubClass_catD,MSZoning_lev_x.FV,MSZoning_lev_x.RL,MSZoning_lev_x.RM,MSZoning_catP,MSZoning_catN,MSZoning_catD,LotFrontage_clean,LotArea_clean,Alley_lev_x.Grvl,Alley_lev_x.NA,Alley_catP,Alley_catN,Alley_catD,LotShape_lev_x.IR1,LotShape_lev_x.IR2,LotShape_lev_x.Reg,LotShape_catP,LotShape_catN,LotShape_catD,LandContour_lev_x.Bnk,LandContour_lev_x.HLS,LandContour_catN,LandContour_catD,LotConfig_lev_x.CulDSac,LotConfig_lev_x.Inside,LotConfig_catP,LotConfig_catN,Neighborhood_lev_x.BrkSide,Neighborhood_lev_x.CollgCr,Neighborhood_lev_x.Crawfor,Neighborhood_lev_x.Edwards,Neighborhood_lev_x.IDOTRR,Neighborhood_lev_x.Mitchel,Neighborhood_lev_x.NAmes,Neighborhood_lev_x.NoRidge,Neighborhood_lev_x.NridgHt,Neighborhood_lev_x.OldTown,Neighborhood_lev_x.Sawyer,Neighborhood_lev_x.Somerst,Neighborhood_lev_x.Timber,Neighborhood_catP,Neighborhood_catN,Neighborhood_catD,Condition1_lev_x.Artery,Condition1_lev_x.Feedr,Condition1_lev_x.Norm,Condition1_catP,Condition1_catN,Condition1_catD,BldgType_lev_x.1Fam,BldgType_lev_x.2fmCon,BldgType_lev_x.Duplex,BldgType_lev_x.Twnhs,BldgType_catP,BldgType_catN,BldgType_catD,HouseStyle_lev_x.1.5Fin,HouseStyle_lev_x.2Story,HouseStyle_lev_x.SFoyer,HouseStyle_catP,HouseStyle_catN,HouseStyle_catD,OverallQual_clean,OverallCond_clean,YearBuilt_clean,YearRemodAdd_clean,RoofStyle_lev_x.Gable,RoofStyle_lev_x.Hip,RoofStyle_catP,RoofStyle_catN,RoofStyle_catD,RoofMatl_lev_x.CompShg,RoofMatl_catP,RoofMatl_catN,Exterior1st_lev_x.Cement,Exterior1st_lev_x.HdBoard,Exterior1st_lev_x.MetalSd,Exterior1st_lev_x.Shingles,Exterior1st_lev_x.VinylSd,Exterior1st_lev_x.Wd.Sdng,Exterior1st_catP,Exterior1st_catN,Exterior1st_catD,Exterior2nd_lev_x.Cement,Exterior2nd_lev_x.MetalSd,Exterior2nd_lev_x.Plywood,Exterior2nd_lev_x.VinylSd,Exterior2nd_lev_x.Wd.Sdng,Exterior2nd_catP,Exterior2nd_catN,Exterior2nd_catD,MasVnrType_lev_x.BrkFace,MasVnrType_lev_x.None,MasVnrType_lev_x.Stone,MasVnrType_catP,MasVnrType_catN,MasVnrType_catD,MasVnrArea_clean,ExterQual_lev_x.3,ExterQual_lev_x.4,ExterQual_lev_x.5,ExterQual_catP,ExterQual_catN,ExterQual_catD,ExterCond_lev_x.Fa,ExterCond_lev_x.TA,ExterCond_catP,ExterCond_catN,ExterCond_catD,Foundation_lev_x.BrkTil,Foundation_lev_x.CBlock,Foundation_lev_x.PConc,Foundation_lev_x.Slab,Foundation_catP,Foundation_catN,Foundation_catD,BsmtQual_lev_x.Ex,BsmtQual_lev_x.Fa,BsmtQual_lev_x.Gd,BsmtQual_lev_x.NA,BsmtQual_lev_x.TA,BsmtQual_catP,BsmtQual_catN,BsmtQual_catD,BsmtCond_lev_x.Fa,BsmtCond_lev_x.Gd,BsmtCond_lev_x.NA,BsmtCond_lev_x.TA,BsmtCond_catP,BsmtCond_catN,BsmtCond_catD,BsmtExposure_lev_x.Av,BsmtExposure_lev_x.Gd,BsmtExposure_lev_x.NA,BsmtExposure_lev_x.No,BsmtExposure_catP,BsmtExposure_catN,BsmtExposure_catD,BsmtFinType1_lev_x.ALQ,BsmtFinType1_lev_x.BLQ,BsmtFinType1_lev_x.GLQ,BsmtFinType1_lev_x.LwQ,BsmtFinType1_lev_x.NA,BsmtFinType1_lev_x.Rec,BsmtFinType1_lev_x.Unf,BsmtFinType1_catP,BsmtFinType1_catN,BsmtFinType1_catD,BsmtFinSF1_clean,BsmtFinType2_lev_x.NA,BsmtFinType2_lev_x.Unf,BsmtFinType2_catP,BsmtFinType2_catN,BsmtFinType2_catD,BsmtUnfSF_clean,TotalBsmtSF_clean,Heating_lev_x.GasA,Heating_catP,Heating_catN,Heating_catD,HeatingQC_lev_x.2,HeatingQC_lev_x.3,HeatingQC_lev_x.4,HeatingQC_lev_x.5,HeatingQC_catP,HeatingQC_catN,HeatingQC_catD,CentralAir_lev_x.N,CentralAir_lev_x.Y,Electrical_lev_x.FuseA,Electrical_lev_x.FuseFP,Electrical_lev_x.SBrkr,Electrical_catP,Electrical_catN,Electrical_catD,X1stFlrSF_clean,X2ndFlrSF_clean,GrLivArea_clean,BsmtFullBath_clean,FullBath_clean,HalfBath_clean,BedroomAbvGr_clean,KitchenAbvGr_clean,KitchenQual_lev_x.2,KitchenQual_lev_x.3,KitchenQual_lev_x.4,KitchenQual_lev_x.5,KitchenQual_catP,KitchenQual_catN,KitchenQual_catD,TotRmsAbvGrd_clean,Functional_lev_x.Min2,Functional_lev_x.Typ,Functional_catP,Functional_catN,Functional_catD,Fireplaces_clean,FireplaceQu_lev_x.Gd,FireplaceQu_lev_x.NA,FireplaceQu_lev_x.TA,Fire

## Defining model object and fitting to private training set

In [4]:
#### Define a linear regression object ols
from sklearn import linear_model
ols = linear_model.LinearRegression()

(Recall from lecture code) 
Some attributes and methods we will use for linear regression:

- `coef_`: Estimated coefficients ($\hat{\beta}_1$,...) for the linear regression problem.
- `intercept_`: Independent term ($\hat{\beta}_0$) in the linear model.
- `fit(X, y)`: Fit linear model.
- `predict(X)`: Predict using the linear model
- `score(X, y)`: Returns the coefficient of determination $R^2$ of the prediction.

In [35]:
# If data isn't already pre-split:
#try:  # train_test_split was moved in 0.18.0
#    from sklearn.model_selection import train_test_split
#except:  # Following import works through 0.19 but outputs a warning in 0.18
#    from sklearn.cross_validation import train_test_split
#X_private_train, X_private_test, yT_private_train, yT_private_test = train_test_split(X_houses_train, yT_houses_train, test_size=0.2, random_state=11)



In [36]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components=200)
#pca.fit(X_private_train)

In [5]:
# Fit a linear model to our private_train data using BoxCoxed response
#ols.fit(pca.transform(X_private_train), yT_private_train)
ols.fit(X_private_train, yT_private_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
# Predict BoxCoxed response outputs
#predictedT = ols.predict(pca.transform(X_private_train))
predictedT = ols.predict(X_private_train)

In [7]:
predictedT

array([ 59.60898702,  56.33606426,  61.58536357, ...,  75.79308148,
        55.50690578,  61.42394931])

In [8]:
# Inverse BoxCox previous prediction to get predicted SalePrices
#predicted = np.power((predictedT * private_lambda) + 1, 1 / private_lambda) 
predicted = np.power((predictedT * 0.22) + 1, 1/(0.22)) # Use line above to use python-found lambda value

In [9]:
# Compute RMSE and R^2 using log of un-BoxCoxed predictions; must use .reshape(-1,1) to match dimensions of object
# containing actual y-values with object containing predicted y-values
print("RMSE: {0}".format(np.sqrt(np.mean((np.log(encoded_private_train['SalePrice'].values) - np.log(predicted)) ** 2))))

RMSE: 0.10210227576270292


## Testing model on private test set

In [10]:
# Predict BoxCoxed response outputs for private test
#test_predictedT = ols.predict(pca.transform(X_private_test))
test_predictedT = ols.predict(X_private_test)

In [12]:
# Inverse BoxCox previous prediction to get predicted SalePrices
test_predicted = np.power((test_predictedT * 0.22) + 1, 1/(0.22))

/Users/kathrynbryant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in power
  


In [13]:
# Compute RMSE and R^2 using log of un-BoxCoxed predictions; must use .reshape(-1,1) to match dimensions of object
# containing actual y-values with object containing predicted y-values
print("RMSE: {0}".format(np.sqrt(np.mean((np.log(encoded_private_test['SalePrice'].values.reshape(-1,1)) - np.log(test_predicted)) ** 2))))

RMSE: nan
